# Домашнее задание №5 - SVM и Naive Bayes classifier

В этом домашнем задании мы с вами напишем реализацию метода опорных векторов (Support Vector Machine), а также посмотрим на наивный байесовский классификатор из sklearn.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import random

from sklearn import datasets
from sklearn.svm import SVC

In [ ]:
plt.rcParams["figure.figsize"] = 12, 9
sns.set_style("whitegrid")

SEED = 111
random.seed(SEED)
np.random.seed(SEED)

## Задание 1. SVM своими руками

**70 баллов**

Идея метода опорных векторов сотоит в нахождении гиперплоскости (в общем нелинейном случае гиперповерхности) в пространстве фичей, оптимально разделяющей заданные объекты с **максимальным** зазором между объектами разных классов. 

Начнем с простого случая: пространство из двух фичей, а гиперплоскость - линейная функция.

### Задание 1.1 Линейный SVM (40 баллов)

Сгенерируем "игрушечные" данные:

In [ ]:
X, y = datasets.make_blobs(n_samples=500, centers=2, random_state=SEED)

plt.figure(figsize=(5, 5))
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
plt.show()

Напишем класс модели `Linear_SVM`, которая подбирает оптимальную разделяющую поверхность (в 2D прямую) для наших данных.

Какой же лосс использовать, чтобы оптимизировать нашу модель? Для SVM (SCV) используют Hinge loss (шарнирная функция потерь). Выглядит она следующим образом:

$$
  J(w, b) = λ \frac{1}{2} ||w||^2 + \frac{1}{n} \sum \limits _{i=1} ^{n} max(0, 1 - y_{i}(w x_{i} + b))
$$


Где w и b - веса и смещения нашей линейной функции. Что мы тут видим:

- Первый член в основном отвечает за максимизацию маржи, выраженной как задача минимизации с добавленным параметром регуляризации λ. Умножение на 1/2 производится исключительно для удобства при вычислении градиента функции.


- Вообще говоря, Hinge Loss это именно второе слогаемое функции выше. Этот член отвечает за то, чтобы мы прогнозировали правильную метку класса с достаточным запасом. Например, если yᵢ = 1 и xᵢ классифицирован правильно, вычисление лосса даст ноль, поскольку max(0, 1–1) = 0. Однако, если метка класса предсказана неверно, лосс даст значение больше нуля.

Мы будем использовать градиентный спуск для оптимизации. Поэтому нам также нужно вычислить производные этой функции. Не будем мучаться больше чем необходимо, ответ приведен ниже. Лосс принимает две формы взависимости от величины линейной функции:

$$J_{i} = λ \frac{1}{2} ||w||^2$$

если $$y_{i}(w x_{i} + b) >= 1$$ и в противном случае

$$
J_{i} = λ \frac{1}{2} ||w||^2 + 1 - y_{i}(w x_{i} + b) 
$$ 

Для этих случаев происзводные принимают вид:

$$
  \frac {\partial J_{i}}{\partial w_{k}} = λ w_{k}
$$

$$
   \frac { \partial J_{i}}{ \partial b} = 0
$$

и

$$
  \frac {\partial J_{i}}{\partial w_{k}} = λ w_{k} - y_{i} x_{i}
$$
$$
   \frac { \partial J_{i}}{ \partial b} = - y_{i}
$$

соответственно.

Перейдем к реализации нашего класса. Вам надо заполнить пропуски в методах `fit`, `estimate` и `predict`, и `Hinge_loss`. 

Для начала реализуйте `estimate` и `predict` основываясь на подсказках в методе. Будьте внимательны относительно типа который возвращают методы.

Затем реализуйте метод `Hinge_loss`. Вам надо воспроизвести формулу выше. Обратите внимание на тип аргументов, принимаемых функцией. Проверьте свою реализацию ниже при помощи assert на паре примеров. Cравните с [функцией из `sklearn`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hinge_loss.html)

Теперь перейдем к `fit`. Алгоритм следующий:

1. Инициализируем весов и смещений


2. Отображаем метки классов из {0, 1} в {-1, 1}


3. Выполняем градиентный спуск `n_iters` итераций. D каждую итерацию проходимся по всем объектам. Для каждого вычисляем градиенты и проводим обновление весов и смещений. На каждой итерации также запоминаем значение функции потерть. Добавим также аргумент `verbose`, который который позволит следить за функцией потерь в ходе обучения.

In [ ]:
class Linear_SVM():
    """
    Linear Support Vector Machine implementation
    
    Attributes
    -----------
    lr: float
        learning rate
        
    lambda_param: float 
        lambda parameter for regularisation
        
    n_iters: int
        numbers of gradient descent iterations
        
    w: np.array(float)
        model weights
    
    b: float
        intercept term in the model
        
    loss_train: np.array(float)
        loss value obtained during fit
    """
    
    def __init__(self, lr=0.001, lambda_param=0.01, n_iters=100):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        
        self.w = None
        self.b = None
        
        self.loss_train = None

    def fit(self, X, y, verbose=False):
        """
        Fit model
        Assign model parameters w and b
        
        X: np.array(n_samples, n_features)
            training data
            
        y: np.array(n_samples)
            class labels, 0 and 1
            
        verbose: bool
            verbose loss during fit
            
        Returns
        ---------
        self
        """
        
        n_samples, n_features = X.shape
        
        # изменим кодировку таргетной переменной на -1, 1,
        y_cls_map = np.where(y <= 0, -1, 1)
        
        # инициализируем веса и смещение
        self.w = np.zeros(n_features)
        self.b = 0
        
        self.loss_train = np.zeros(self.n_iters) # oбнуляем лоссы
        for n in range(self.n_iters):
            
            # в ходе одной итерации проходимся по всем 
            # объектам в трейне
            for i, x_i in enumerate(X):
                
                # для каждого объекта проверяем правильность предсказания
                estimate = self.estimate(x_i) 
                condition = (y_cls_map[i] * estimate) >= 1
                
                # вычисляем градиент
                # если объект на верной стороне гиперплоскости
                if condition:
                    dw = #YOUR CODE HERE
                    db = #YOUR CODE HERE
                # если объект на неверной стороне гиперплоскости
                else:
                    dw = #YOUR CODE HERE
                    db = #YOUR CODE HERE
                
                # обновляем веса
                self.w -= self.lr * dw
                self.b -= self.lr * db
                
                # рассчитываем и выводим лосс
                loss = #YOUR CODE HERE
                self.loss_train[n] = loss
                if verbose:
                    print(f"--------------- Epoch {n + 1} --> Loss = {round(loss, 3)} ---------------")
            
        return self
    
    def Hinge_loss(self, y, y_pred):
        """
        Hinge loss
        
        y: np.array(int)
            true labels, 0 and 1
            
        y_pred: np.array(int)
            predicted labels, 0 and 1
        
        Returns
        ---------
        float, loss value
        """
        
        # YOUR CODE HERE
        
        return 
    
    def estimate(self, X):
        """
        Вычисляем и возвращаем значение
        линейной функции wX + b
        
        X: np.array(n_samples, n_features)
            training data
            
        Returns
        ---------
        np.array(float)
        """
        return # YOUR CODE HERE
    
    def predict(self, X):
        """
        Предсказываем метки классов
        
        X: np.array(n_samples, n_features)
            training data
            
         Returns
        ---------
        np.array(int) array of class labels as 0 and 1
        """
        
        # сначала вычисляем self.estimate(X) и конвертируем значения в метки классов
        # не забудьте, что сначала мы определяем класс как -1/1 сравнивая с 0
        # а потом трансформируем в 0/1
        
        # YOUR CODE HERE
        
        return 

Обучите свою модель на сгенерированных выше данных. Разбивать на трейн и тест не надо. Постройте график лосс функции. 

In [ ]:
# обучаем модель
lin_SVM = Linear_SVM()
lin_SVM.fit(X, y)

In [ ]:
# строим график лосса

Визуализируйте, как модель разделила данные, используя заготовленную ниже функцию `plot_decision_boundary`. Обратите внимание, что фурнкция строит границу раздела численно, а не аналитически, поэтому у вас будет немного ломанная линия, а не идеальная прямая, это нормально.

In [ ]:
def plot_decision_boundary(X, y, model):
    
    plt.figure(figsize=(5, 5))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr')
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50))
    xy = np.vstack([xx.ravel(), yy.ravel()]).T
    Z = model.predict(xy).reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='bwr')
    plt.show()

In [ ]:
plot_decision_boundary(X, y, lin_SVM)

Посмотрите на получившийся график, возьмите 5 произвольных точекс плоскости (не из Х, а на глаз), которые будут принадлежать к разным классам и используйте их как пример для предсказания:

In [ ]:
# пример предсказания
X_test = 

В реальных случаях возможность эффективно разделить оъекты линейной функцией - довольно редкий случай. Усложним наш игрушечный датасет. Проверьте, как модель `Linear_SVM` справится с этими данными. 


Обучите модель и постройте график лосса и границу раздела. Вычислите метрики класификации (accuracy, precision, recall, f1) на обучающей выборке.

In [ ]:
X, y = datasets.make_moons(n_samples=500, noise=0.30, random_state=SEED)

plt.figure(figsize=(8, 6))
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
plt.show()

In [ ]:
# обучаем модель
lin_SVM = Linear_SVM()
lin_SVM.fit(X, y)

In [ ]:
plot_decision_boundary(X, y, lin_SVM)

In [ ]:
# строим график лосса

In [ ]:
№

### Задание 1.2 SVM с нелинейным ядром (30 баллов)

Чтобы найти нелинейную границу решения, мы можем добавить такой прием как «ядро» (kernel). Если данные не являются линейно разделимыми в исходном пространстве, то мы применяем преобразованиe к данным, которое отображает данные из исходного пространства в более многомерное пространство. Цель состоит в том, что после преобразования в более многомерное пространство классы становятся линейно разделимыми. Кажется не очевидно, но вот вам картинка с визуальным примером перехода из двумерного пространства в трехмерное:

![](kernel_transformation.png)

Таким образом наша линейная функция wX + b превращается вследующее:

$$
  \sum \limits _{i} α_{i} K(x_{i}, x) + b
$$

где K(xi, x) это kernel функция, описфвающая трансформацию наших данных. Давайте возьмем SVC из sklearn и посмотрим как разные ядра, справятся с разделением модельных данных. попробуйте 'rbf', 'poly' (посмотрите на разные степени) и 'sigmoid'. Для якаждого случая посчитайте метрики классификациии на данных использованных для обучения и постройте границу раздела. Опишите, что получилось.

доп. 10 баллов, если оформите все графики в красивый мультиплот с несколькими колонками и рядами.

In [ ]:
clf = SVC(kernel= ???)
clf.fit(X, y)

In [ ]:
# ваш ход

## Задание 2. Применение баесовского классификатора

**80 баллов**

Мы будем использовать данные по свойствам покемонов (https://www.kaggle.com/abcsds/pokemon). 

In [ ]:
pokemon = pd.read_csv("Pokemon.csv")
pokemon.head()

Начинаем конечно же с EDA. (35 баллов) Посмотрите на данные, типы фичей, их описательные статистики и распределения. А также также посмотреть, как различные признаки связаны между собой и с целевой переменной `Legendary` (постройте хитмап корреляции между фичами и целевой переменной). 

Мы будем предсказывать является ли покемон легендарным или нет. Замените логическое значение колонки `Legendary` на числовое (перекодировав на 0 и 1). 

**Вопрос: как в этом случае лучше закодировать категориальные признаки (может быть, лучше их просто выбросить?).**

...

Разделите ваши данные на тестовую и тренировочную выборку.

Обучите модель `GaussianNB` из `sklearn` (5 баллов). Побробнее про разные реализации NB в sklearn можно почитать [тут](https://scikit-learn.org/stable/modules/naive_bayes.html)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
## ENTER YOUR CODE HERE (/¯◡ ‿ ◡)/¯☆*##

Поработаем с получившейся моделью:

1. Посчитайте метрики классификации (7 баллов)

2. Нарисуйте confusion matrix (8 баллов)

3. Изобразите ROC кривую и посчитайте площадь под ней (10 баллов)

4. Скажите, какие признаки оказались наиболее важны для модели? (10 баллов)

In [ ]:
## ENTER YOUR CODE HERE (/¯◡ ‿ ◡)/¯☆*##

## Therapy time

Напишите здесь ваши впечатления о задании: было ли интересно, было ли слишком легко или наоборот сложно и тд. Также сюда можно написать свои идеи по улучшению заданий, а также предложить данные, на основе которых вы бы хотели построить следующие дз. 

**Ваши мысли:**